In [10]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
import joblib
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
import subprocess
from sklearn.utils import shuffle
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import MinMaxScaler
import itertools
name="PseEIIP_RFHC_GAC_PCP"

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall==0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

"""
    cross validation
"""

datapath1 ="/home02/chenhuangrong/trying_merge_PseEIIP_RFHC_GAC_PCP/PCP.csv"
datapath2="/home02/chenhuangrong/trying_merge_PseEIIP_RFHC_GAC_PCP/PseEIIP.csv"
datapath3="/home02/chenhuangrong/trying_merge_PseEIIP_RFHC_GAC_PCP/GAC_RFH.csv"
PCP_data=pd.read_csv(datapath1,header=None,index_col=None)
PCP_data=PCP_data.values
print PCP_data.shape
PseEIIP_data=pd.read_csv(datapath2,header=None,index_col=None)
PseEIIP_data=PseEIIP_data.values
print PseEIIP_data.shape
GAC_RFH_data=pd.read_csv(datapath2,header=None,index_col=None)
GAC_RFH_data=GAC_RFH_data.values
print GAC_RFH_data.shape

data=np.concatenate([PCP_data,PseEIIP_data,GAC_RFH_data],axis=1)
train_data=data
print train_data
print train_data.shape
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
pval,p=f_classif(X,Y)
pval=np.argsort(pval)[::-1]
for index in xrange(len(X[0])):
    X=train_data[:,0:(index+1)]
    scaler=MinMaxScaler()
    X=scaler.fit_transform(X)
    print X.shape
    print train_data.shape
    svc = svm.SVC()
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=-1, scoring='accuracy')
    clf.fit(X, Y)
    C=clf.best_params_['C']
    # joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
    gamma=clf.best_params_['gamma']
    y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=4)
    ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=4).mean()
    y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=4,method='predict_proba')
    predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
    predict_save=np.array(predict_save).T
    pd.DataFrame(predict_save).to_csv(name+'_predict.csv',header=None,index=False)
    ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
    ACC=metrics.accuracy_score(Y,y_predict)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
    F1_Score=metrics.f1_score(Y, y_predict)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y, y_predict)
    pos=TP+FN
    neg=FP+TN
    savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
    print savedata
    print X.shape[1]
    easy_excel.save("svm_crossvalidation",[str(X.shape[1])],savedata,'/home02/chenhuangrong/trying_merge_PseEIIP_RFHC_GAC_PCP/c'+name+'.xls')


(2614, 400)
(2614, 64)
(2614, 64)
[[-0.22308202 -0.19825363  0.22347011 ...,  0.00601429  0.          0.00493469]
 [-0.07503061 -0.19552073  0.14951684 ...,  0.          0.          0.00493469]
 [-0.00700908 -0.12901633  0.07967889 ...,  0.00601429  0.          0.        ]
 ..., 
 [-0.21004156 -0.29398209  0.20550607 ...,  0.          0.00602449  0.        ]
 [-0.03431849 -0.02907642  0.09443612 ...,  0.          0.          0.        ]
 [ 0.11219506 -0.08745316  0.08058142 ...,  0.00601429  0.          0.        ]]
(2614, 528)
(2614, 1)
(2614, 528)
0.520657995409
[[['svmC:14.2543794902gamma:0.793700525984', 0.52065799540933433, 0.5210608424336973, 0.5110941086457537, 0.5110941086457537, 0.5302218821729151, 0.520570149214916, 0.51602935496330637, 0.51602935496330637, 0.041323551069607868, '0.5206', 668.0, 639.0, 614.0, 693.0, 1307.0, 1307.0]]]
1
(2614, 2)
(2614, 528)
0.517214996174
[[['svmC:1.25992104989gamma:0.157490131237', 0.51721499617444533, 0.5163398692810458, 0.5439938791124713,